In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

In [2]:
# 如果使用 xgboost
try:
    import xgboost as xgb
    has_xgb = True
except Exception:
    has_xgb = False
    print("未检测到 xgboost，若需使用请安装：pip install xgboost")

In [3]:
# ---------- 配置 ----------
DATA_PATH = "E:/ML/wine_quality/data/winequality-red.csv"  # 改成你的路径
RANDOM_STATE = 42
TEST_SIZE = 0.2

os.makedirs("results/figures", exist_ok=True)
os.makedirs("results/metrics", exist_ok=True)

In [4]:
# ---------- 1. 加载数据 ----------
df = pd.read_csv(DATA_PATH, sep=';')  # UCI 葡萄酒数据集通常分号分隔
print("数据形状:", df.shape)
print(df.head())

数据形状: (1599, 12)
   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2  

In [5]:
# ---------- 2. 初步可视化（分布 & 相关性） ----------
# 基本统计
print(df.describe())

# 目标分布
plt.figure(figsize=(6,4))
plt.hist(df['quality'], bins=range(int(df['quality'].min()), int(df['quality'].max())+2), rwidth=0.8)
plt.xlabel("quality")
plt.ylabel("count")
plt.title("Quality distribution")
plt.savefig("results/figures/quality_distribution.png", bbox_inches='tight')
plt.close()

# 相关矩阵热力图（使用 pandas）
corr = df.corr()
plt.figure(figsize=(10,8))
plt.matshow(corr)
plt.colorbar()
plt.title("Correlation matrix (visual)", y=1.2)
plt.savefig("results/figures/correlation_matrix.png", bbox_inches='tight')
plt.close()


       fixed acidity  volatile acidity  citric acid  residual sugar  \
count    1599.000000       1599.000000  1599.000000     1599.000000   
mean        8.319637          0.527821     0.270976        2.538806   
std         1.741096          0.179060     0.194801        1.409928   
min         4.600000          0.120000     0.000000        0.900000   
25%         7.100000          0.390000     0.090000        1.900000   
50%         7.900000          0.520000     0.260000        2.200000   
75%         9.200000          0.640000     0.420000        2.600000   
max        15.900000          1.580000     1.000000       15.500000   

         chlorides  free sulfur dioxide  total sulfur dioxide      density  \
count  1599.000000          1599.000000           1599.000000  1599.000000   
mean      0.087467            15.874922             46.467792     0.996747   
std       0.047065            10.460157             32.895324     0.001887   
min       0.012000             1.000000         

<Figure size 1000x800 with 0 Axes>

In [6]:
# ---------- 3. 处理缺失值 ----------
# 检查是否有缺失
print("缺失值统计：\n", df.isnull().sum())
# 示例策略：如果有少量缺失用中位数填充；如果很多可考虑删除列/行
df = df.fillna(df.median())  # 简单的中位数填充

缺失值统计：
 fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64


In [7]:
# ---------- 4. 特征与目标分离 ----------
X = df.drop(columns=['quality'])
y = df['quality']

# 9/29：做特征缩放（StandardScaler）
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 保存 scaler 以备后用
joblib.dump(scaler, "E:/ML/wine_quality/results/scaler.joblib")

['E:/ML/wine_quality/results/scaler.joblib']

In [8]:
# ---------- 5. 划分训练/测试集 ----------
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
print("训练集大小:", X_train.shape, "测试集大小:", X_test.shape)

训练集大小: (1279, 11) 测试集大小: (320, 11)


In [9]:
# ---------- 6. 训练 Baseline：Linear Regression ----------
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

def evaluate_model(y_true, y_pred, prefix="model"):
    # 兼容旧版本scikit-learn：使用np.sqrt计算RMSE
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{prefix} -> RMSE: {rmse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}")
    return {"rmse": rmse, "mae": mae, "r2": r2}

metrics_lr = evaluate_model(y_test, y_pred_lr, prefix="LinearRegression")
joblib.dump(lr, "results/lr_model.joblib")

# 保存残差图（预测 vs 真实）
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred_lr, alpha=0.6)
plt.xlabel("True quality")
plt.ylabel("Predicted quality")
plt.title("LinearRegression: Pred vs True")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--')
plt.savefig("results/figures/pred_vs_true_lr.png", bbox_inches='tight')
plt.close()

LinearRegression -> RMSE: 0.6245, MAE: 0.5035, R2: 0.4032


In [10]:
# ---------- 7. 训练 RandomForest ----------
rf = RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
metrics_rf = evaluate_model(y_test, y_pred_rf, prefix="RandomForest")
joblib.dump(rf, "results/rf_model.joblib")

# 特征重要性（原始特征名）
feat_importances = rf.feature_importances_
feat_names = X.columns
fi_df = pd.DataFrame({"feature": feat_names, "importance": feat_importances}).sort_values(by="importance", ascending=False)
fi_df.to_csv("results/metrics/rf_feature_importance.csv", index=False)
print(fi_df)

# 绘图
plt.figure(figsize=(8,5))
plt.barh(fi_df['feature'][::-1], fi_df['importance'][::-1])
plt.xlabel("importance")
plt.title("Random Forest Feature Importance")
plt.tight_layout()
plt.savefig("results/figures/rf_feature_importance.png", bbox_inches='tight')
plt.close()

RandomForest -> RMSE: 0.5490, MAE: 0.4220, R2: 0.5389
                 feature  importance
10               alcohol    0.270868
9              sulphates    0.148406
1       volatile acidity    0.111547
6   total sulfur dioxide    0.076786
4              chlorides    0.071132
8                     pH    0.061418
3         residual sugar    0.057892
0          fixed acidity    0.053186
7                density    0.050816
2            citric acid    0.050752
5    free sulfur dioxide    0.047197


In [11]:
# ---------- 8. 交叉验证（KFold）示例 ----------
kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
cv_scores = cross_val_score(rf, X_scaled, y, cv=kf, scoring='neg_root_mean_squared_error', n_jobs=-1)
print("RandomForest 5-fold CV RMSE (negated):", cv_scores)
print("平均 CV RMSE:", -np.mean(cv_scores))

RandomForest 5-fold CV RMSE (negated): [-0.5608264  -0.59628485 -0.59896917 -0.5820473  -0.54495147]
平均 CV RMSE: 0.5766158373223337


In [ ]:
# ---------- 9. XGBoost 训练与交叉验证（如果安装了 xgboost） ----------
if has_xgb:
    # 原始XGBoost（默认参数）
    print("=== 训练原始XGBoost模型 ===")
    xgbr_default = xgb.XGBRegressor(objective='reg:squarederror', random_state=RANDOM_STATE, n_jobs=-1)
    xgbr_default.fit(X_train, y_train)
    y_pred_xgb_default = xgbr_default.predict(X_test)
    metrics_xgb_default = evaluate_model(y_test, y_pred_xgb_default, prefix="XGBoost (默认参数)")
    
    # 调优后的XGBoost（针对小数据集优化）
    print("\n=== 训练调优XGBoost模型 ===")
    xgbr_tuned = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=50,           # 减少树的数量防止过拟合
        max_depth=4,               # 限制深度
        learning_rate=0.1,         # 学习率
        subsample=0.8,             # 子采样
        colsample_bytree=0.8,      # 特征采样
        reg_alpha=0.1,             # L1正则化
        reg_lambda=1.0,            # L2正则化
        random_state=RANDOM_STATE,
        n_jobs=-1
    )
    xgbr_tuned.fit(X_train, y_train)
    y_pred_xgb_tuned = xgbr_tuned.predict(X_test)
    metrics_xgb_tuned = evaluate_model(y_test, y_pred_xgb_tuned, prefix="XGBoost (调优参数)")
    
    # 使用调优后的模型作为最终结果
    metrics_xgb = metrics_xgb_tuned
    y_pred_xgb = y_pred_xgb_tuned
    
    # 保存模型
    joblib.dump(xgbr_tuned, "results/xgb_model_tuned.joblib")
    
    # XGBoost特征重要性分析
    print("\n=== XGBoost特征重要性 ===")
    feature_names = df.drop(columns=['quality']).columns
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': xgbr_tuned.feature_importances_
    }).sort_values('importance', ascending=False)
    print(importance_df)
    
    # XGBoost交叉验证
    print("\n=== XGBoost 5折交叉验证 ===")
    xgb_cv_scores = cross_val_score(xgbr_tuned, X_scaled, y, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
    print(f"XGBoost CV RMSE scores: {xgb_cv_scores}")
    print(f"平均 CV RMSE: {-np.mean(xgb_cv_scores):.4f}")
    
else:
    metrics_xgb = None
    print("XGBoost未安装，跳过训练")

XGBoost -> RMSE: 0.5927, MAE: 0.4175, R2: 0.4625


In [13]:
# ---------- 10. GridSearchCV 示例（对 RandomForest 调参，作为 10/4 的工作起点） ----------
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 8, 12],
    'min_samples_split': [2, 5]
}
gsearch = GridSearchCV(RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1),
                       param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1)
gsearch.fit(X_train, y_train)
print("GridSearch 最佳参数：", gsearch.best_params_)
best_rf = gsearch.best_estimator_
y_pred_best_rf = best_rf.predict(X_test)
metrics_best_rf = evaluate_model(y_test, y_pred_best_rf, prefix="RF (GridSearch best)")
joblib.dump(gsearch, "results/rf_gridsearch.joblib")


Fitting 3 folds for each of 12 candidates, totalling 36 fits
GridSearch 最佳参数： {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
RF (GridSearch best) -> RMSE: 0.5490, MAE: 0.4220, R2: 0.5389


['results/rf_gridsearch.joblib']

In [ ]:
# ---------- 11. 保存最终评估表（包含详细指标解释） ----------
results_summary = {
    "LinearRegression": metrics_lr,
    "RandomForest": metrics_rf,
    "RandomForest_GridSearch": metrics_best_rf,
    "XGBoost": metrics_xgb
}

# 创建详细的评估报告
print("=== 模型性能详细对比 ===")
print("指标说明：")
print("- RMSE: 均方根误差，越小越好，单位与目标变量相同")
print("- MAE: 平均绝对误差，越小越好，对异常值不敏感")
print("- R²: 决定系数，越接近1越好，表示模型解释数据变异的能力")
print()

# 打印对比表格
comparison_data = []
for model_name, metrics in results_summary.items():
    if metrics is not None:
        comparison_data.append({
            "模型": model_name,
            "RMSE": f"{metrics['rmse']:.4f}",
            "MAE": f"{metrics['mae']:.4f}",
            "R²": f"{metrics['r2']:.4f}"
        })

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

# 找出最佳模型
best_model = max(results_summary.items(), key=lambda x: x[1]['r2'] if x[1] is not None else 0)
print(f"\n🏆 最佳模型: {best_model[0]}")
print(f"   R² = {best_model[1]['r2']:.4f} (能解释{best_model[1]['r2']*100:.1f}%的数据变异)")
print(f"   RMSE = {best_model[1]['rmse']:.4f} (平均预测误差)")
print(f"   MAE = {best_model[1]['mae']:.4f} (平均绝对误差)")

# 保存为CSV
rows = []
for k,v in results_summary.items():
    if v is None:
        continue
    rows.append({"model": k, "rmse": v["rmse"], "mae": v["mae"], "r2": v["r2"]})
pd.DataFrame(rows).to_csv("results/metrics/model_comparison.csv", index=False)
print("\n已保存 model_comparison.csv")

已保存 model_comparison.csv


In [15]:
# ---------- 12. 简单的残差直方图（以最佳 RF 为例） ----------
residuals = y_test - y_pred_best_rf
plt.figure(figsize=(6,4))
plt.hist(residuals, bins=30)
plt.xlabel("residual")
plt.title("Residuals histogram (best RF)")
plt.savefig("results/figures/residuals_best_rf.png", bbox_inches='tight')
plt.close()

print("流程完成。检查 results/ 下的输出文件并上传至 GitHub。")

流程完成。检查 results/ 下的输出文件并上传至 GitHub。


In [ ]:
# ---------- 13. 模型性能可视化分析 ----------
import matplotlib.pyplot as plt

# 创建模型性能对比图
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. RMSE对比
models = ['LinearRegression', 'RandomForest', 'XGBoost']
rmse_values = [metrics_lr['rmse'], metrics_rf['rmse'], metrics_xgb['rmse']]
axes[0,0].bar(models, rmse_values, color=['skyblue', 'lightgreen', 'lightcoral'])
axes[0,0].set_title('RMSE对比 (越小越好)', fontsize=14)
axes[0,0].set_ylabel('RMSE')
for i, v in enumerate(rmse_values):
    axes[0,0].text(i, v + 0.01, f'{v:.4f}', ha='center', va='bottom')

# 2. MAE对比
mae_values = [metrics_lr['mae'], metrics_rf['mae'], metrics_xgb['mae']]
axes[0,1].bar(models, mae_values, color=['skyblue', 'lightgreen', 'lightcoral'])
axes[0,1].set_title('MAE对比 (越小越好)', fontsize=14)
axes[0,1].set_ylabel('MAE')
for i, v in enumerate(mae_values):
    axes[0,1].text(i, v + 0.01, f'{v:.4f}', ha='center', va='bottom')

# 3. R²对比
r2_values = [metrics_lr['r2'], metrics_rf['r2'], metrics_xgb['r2']]
axes[1,0].bar(models, r2_values, color=['skyblue', 'lightgreen', 'lightcoral'])
axes[1,0].set_title('R²对比 (越接近1越好)', fontsize=14)
axes[1,0].set_ylabel('R²')
axes[1,0].set_ylim(0, 1)
for i, v in enumerate(r2_values):
    axes[1,0].text(i, v + 0.02, f'{v:.4f}', ha='center', va='bottom')

# 4. 综合性能雷达图
from math import pi
categories = ['RMSE\n(反向)', 'MAE\n(反向)', 'R²\n(正向)']
N = len(categories)

# 标准化数据 (RMSE和MAE越小越好，所以用1-标准化值)
rmse_norm = [1 - (v - min(rmse_values)) / (max(rmse_values) - min(rmse_values)) for v in rmse_values]
mae_norm = [1 - (v - min(mae_values)) / (max(mae_values) - min(mae_values)) for v in mae_values]
r2_norm = r2_values

angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

# 绘制每个模型的雷达图
colors = ['skyblue', 'lightgreen', 'lightcoral']
for i, model in enumerate(models):
    values = [rmse_norm[i], mae_norm[i], r2_norm[i]]
    values += values[:1]
    
    axes[1,1].plot(angles, values, 'o-', linewidth=2, label=model, color=colors[i])
    axes[1,1].fill(angles, values, alpha=0.25, color=colors[i])

axes[1,1].set_xticks(angles[:-1])
axes[1,1].set_xticklabels(categories)
axes[1,1].set_ylim(0, 1)
axes[1,1].set_title('综合性能雷达图', fontsize=14)
axes[1,1].legend()
axes[1,1].grid(True)

plt.tight_layout()
plt.savefig("results/figures/model_performance_comparison.png", dpi=300, bbox_inches='tight')
plt.show()

print("模型性能对比图已保存为 results/figures/model_performance_comparison.png")
